In [1]:
library(keras)
use_condaenv("r-tensorflow")
use_session_with_seed(7)
options(keras.view_metrics = TRUE)

Set session seed to 7 (disabled GPU)


# A first look at a neural network

This notebook contains the code samples found in Chapter 2, Section 1 of [Deep Learning with R](https://www.manning.com/books/deep-learning-with-r). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

We will now take a look at a first concrete example of a neural network, which makes use of the Python library Keras to learn to classify 
hand-written digits. Unless you already have experience with Keras or similar libraries, you will not understand everything about this 
first example right away. You probably haven't even installed Keras yet. Don't worry, that is perfectly fine. In the next chapter, we will 
review each element in our example and explain them in detail. So don't worry if some steps seem arbitrary or look like magic to you! 
We've got to start somewhere.

The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 
categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been 
around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test 
images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. You can think of "solving" MNIST 
as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected. As you become a machine 
learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.

The MNIST dataset comes preloaded in Keras, in the form of `train` and `test` lists, each of which includes a set of images (`x`) and associated labels (`y`):

In [ ]:
# loading the data and creating the train and test sets
mnist <- dataset_mnist()
train_images <- mnist$train$x
train_labels <- mnist$train$y
test_images <- mnist$test$x
test_labels <- mnist$test$y

<b> <center> Lets take a look at some of the images and their corresponding labels

In [ ]:
image_number = 66  # <=====   Change me to see the different digits and their labels


options(repr.plot.width=2, repr.plot.height=2)
par(oma=c(0,0,0,0), mar=c(0,0,2,0))
img  <-  t(apply(as.matrix(train_images[image_number, , ]), 2, rev))
image(img, asp=1, axes=FALSE)
title(main = paste("Digit: ",train_labels[image_number]), cex.main=0.7)

## We can take a look at a few more of the images

In [ ]:
start_index = 14  # <==========  Change me to see a different set of digits and their labels
row = 4
col = 4

options(repr.plot.width=4, repr.plot.height=4)
par(mfrow=c(row,col), mar=c(0.1,0.1,0.1,0.1))
for(i in seq(row*col)){
    index = start_index + i
    image_1 <- as.data.frame(test_images[index, , ])
    m  <-  t(apply(as.matrix(image_1), 2, rev))
    image(m, asp=1, axes=FALSE) 
}


`train_images` and `train_labels` form the "training set", the data that the model will learn from. The model will then be tested on the 
"test set", `test_images` and `test_labels`. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging 
from 0 to 9. There is a one-to-one correspondence between the images and the labels.

<center><h3>Let's have a look at the training data:

In [ ]:
dim(train_images)

In [ ]:
length(train_labels)

In [ ]:
dim(train_labels)

<center><h3>Let's have a look at the test data:

In [ ]:
dim(test_images)

In [ ]:
length(test_labels)

In [ ]:
str(test_labels)

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The 
network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we 
will verify if these predictions match the labels from `test_labels`.

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

In [ ]:
#linear stack of layers
network <- keras_model_sequential() 

# The model needs to know what input shape it should expect. 
# For this reason, the first layer in a sequential model 
# (and only the first, because following layers can do automatic shape inference) 
# needs to receive information about its input shape.
network %>%
  layer_dense(units = 128, activation = 'relu', input_shape = (28 * 28)) %>%
  #layer_dropout(rate = 0.4) %>%
  layer_dense(units = 10, activation = 'softmax')


# network = models.Sequential()
# network.add(layers.Dense(128, activation='relu', input_shape=(28 * 28,)))
# network.add(layers.Dense(10, activation='softmax'))


The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some 
data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully 
representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers 
which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a 
succession of increasingly refined data filters -- the "layers".

Here our network consists of a sequence of two `Dense` layers, which are densely-connected (also called "fully-connected") neural layers. 
The second (and last) layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each 
score will be the probability that the current digit image belongs to one of our 10 digit classes.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be 
able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly 
classified).

The exact purpose of the loss function and the optimizer will be made clear throughout the next two chapters.

In [ ]:
#Next, compile the model with appropriate loss function, optimizer, and metrics:
network %>% compile(
  loss = 'categorical_crossentropy',
  optimizer = optimizer_rmsprop(),
  metrics = c('accuracy')
)


In [ ]:
summary(network)


Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in 
the `[0, 1]` interval. Previously, our training images for instance were stored in an array of shape `(60000, 28, 28)` of type `uint8` with 
values in the `[0, 255]` interval. We transform it into a `float32` array of shape `(60000, 28 * 28)` with values between 0 and 1.

In [ ]:
# reshape the 3D data into a 2D array by flattening the 28 by 28 width and height dimension)
# Note that we use the array_reshape() function rather than the dim<-() function to reshape 
# the array. This is so that the data is re-interpreted using row-major semantics 
# (as opposed to R’s default column-major semantics), which is in turn compatible with the 
# way that the numerical libraries called by Keras interpret array dimensions.
# we also rescale the image greyscale to be between 0 and 1
train_images <- array_reshape(train_images, c(nrow(train_images), (28 * 28)))
train_images <- train_images / 255

test_images <- array_reshape(test_images, c(nrow(test_images), (28 * 28)))
test_images <- test_images / 255


We also need to categorically encode the labels, a step which we explain in chapter 3:

In [ ]:
# prepping the y data by turning them into a binary class using one-hot-encode
train_labels <- to_categorical(train_labels, 10)
test_labels <- to_categorical(test_labels, 10)

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

In [ ]:
# history = network.fit(train_images, train_labels, epochs=15, batch_size=128)

history <- network %>% fit(
    train_images, train_labels,
    epochs = 15, batch_size = 128,
    validation_split = 0.2
)


<center><h3> We can look at the training history 

In [ ]:
history

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
plot(history)

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over 
the training data.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data. Now let's check that our model performs well on the test set too:

In [ ]:
network %>% evaluate(test_images, test_labels)


Our test set accuracy turns out to be 97.8% -- that's quite a bit lower than the training set accuracy. 
This gap between training accuracy and test accuracy is an example of "overfitting", 
the fact that machine learning models tend to perform worse on new data than on their training data. 
Overfitting will be a central topic in chapter 3.

This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits, in 
less than 20 lines of Python code. In the next chapter, we will go in detail over every moving piece we just previewed, and clarify what is really 
going on behind the scenes. You will learn about "tensors", the data-storing objects going into the network, about tensor operations, which 
layers are made of, and about gradient descent, which allows our network to learn from its training examples.

## Compare the predicted digit value with the labelled data

In [ ]:
classes = network %>% predict_classes(test_images)

In [ ]:
valid_class = 15   # <============== Change me to see different predictions.

# extract the prediction, we created a 10 class matrix earlier, so we need to figure out
# which class was predicted for the chosen image (ie which entry is 1 as opposed to 0)
predict = which.max((test_labels)[valid_class,])-1
m <- (matrix(rev(test_images[valid_class,]),nrow = 28,ncol = 28))

# Plotting the test image
options(repr.plot.width=2, repr.plot.height=2)
par(oma=c(0,0,0,0), mar=c(0,0,2,0))
image(m, asp=1, axes=FALSE,xlim=c(1,0))
title(main =  paste("Predicted:",classes[valid_class],"| Actual:",predict), cex.main=0.9)